# Adaptation des données

Nous avons maintenant un model performant et une idée d'un nombre de feature efficace. Mais il faut maintenant faire un protocole concis qui va pouvoir utiliser de nouvelle donnée non-formaté à notre model.


In [3]:
import numpy as np
import pandas as pd #Lecture de notre dataset test
from sklearn.ensemble import RandomForestClassifier #Notre model
import pickle #La sauvegarde de notre model entrainé 
from sklearn.preprocessing import scale #Normalisation des données
from sklearn.preprocessing import OneHotEncoder, LabelEncoder #encodage des données qualitative




In [ ]:

model=pickle.load(open('model.pkl', 'rb'))

### Nos données

In [96]:
df=pd.read_csv("test.csv")

In [97]:
### Notre data de base on peut s'attendre à avoir toujours le même genre de data
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural


### La liste de nos features importante retenu pour le modèle

In [98]:
Best_feat=['Never_Credit','Total_Income', 'Refund_Time','Income_pond','ApplicantIncome','CoapplicantIncome', 'LoanAmount',"Loan_Amount_Term"]

In [99]:
Best_feat

['Never_Credit',
 'Total_Income',
 'Refund_Time',
 'Income_pond',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term']

On a donc 8 variable dont 3 qu'il va falloir recréés : Déjà on drop ce qui ne va pas nous servir.

In [100]:
df.drop(["Loan_ID","Gender","Education","Property_Area","Self_Employed"],axis= 1, inplace=True)

In [101]:
df 

,Married,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,Yes,0,5720,0,110.0,360.0,1.0
1,Yes,1,3076,1500,126.0,360.0,1.0
2,Yes,2,5000,1800,208.0,360.0,1.0
3,Yes,2,2340,2546,100.0,360.0,NaN
4,No,0,3276,0,78.0,360.0,1.0
...,...,...,...,...,...,...,...
362,Yes,3+,4009,1777,113.0,360.0,1.0
363,Yes,0,4158,709,115.0,360.0,1.0
364,No,0,3250,1993,126.0,360.0,NaN
365,Yes,0,5000,2393,158.0,360.0,1.0


On encode nos catégories on vire nos NaN et on créer nos nouvelles features

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Married            367 non-null    object 
 1   Dependents         357 non-null    object 
 2   ApplicantIncome    367 non-null    int64  
 3   CoapplicantIncome  367 non-null    int64  
 4   LoanAmount         362 non-null    float64
 5   Loan_Amount_Term   361 non-null    float64
 6   Credit_History     338 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 20.2+ KB


#### On doit enlever des NaN pour encoder

In [103]:
df['Loan_Amount_Term'].loc[df["Loan_Amount_Term"].isnull()] = 360

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [104]:
df['Dependents'].loc[df['Dependents'].isnull()]= "0"

In [105]:
le=LabelEncoder()

df["Married"]=le.fit_transform(y=df["Married"])
df['Dependents']=le.fit_transform(y=df['Dependents'])
    

#### On crée nos nouvelles variables: 

<ul><li>Total Income = ApplicantIncome + CoaaplicantIncome</li>
    <li>Income_pondéré= Total_Income / (dependent + 1 if married)</li>
    <li>Mois_d'échéance= LoanAmount / Loan_Amount_Term</li></ul>

In [107]:
df['LoanAmount']=df["LoanAmount"]*100
df['Total_Income']=df["ApplicantIncome"]+df['CoapplicantIncome']
df["Refund_Time"]= round(df["LoanAmount"]/df['Loan_Amount_Term'])

In [108]:
df['Family']= np.where(df['Married']==0, 1+df['Dependents'],2+df['Dependents'])

#On divise l'income total dans par ce chiffre. 
df['Income_pond']=df['Total_Income']/df['Family']

In [113]:
df= df[['Credit_History',
 'Total_Income',
 'Refund_Time',
 'Income_pond',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term']]

In [117]:
df.dropna(axis=0,how="any", inplace=True)

In [135]:
ohe= OneHotEncoder(sparse=False)
integer_encoded= df[["Credit_History"]]
X= ohe.fit_transform(integer_encoded)

In [138]:
df[["Never_Credit","Already_Credit"]]=X

In [141]:
df.drop(["Credit_History","Already_Credit"],axis=1,inplace=True)

In [143]:
df = df[['Never_Credit',
 'Total_Income',
 'Refund_Time',
 'Income_pond',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term']]

#### On scale

In [144]:
df

,Never_Credit,Total_Income,Refund_Time,Income_pond,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
0,0.0,-0.107408,-0.153022,-0.157753,0.201334,-0.668038,-0.451727,0.277302
1,0.0,-0.329947,-0.125622,-0.570858,-0.346186,-0.010695,-0.177947,0.277302
2,0.0,0.102680,0.031925,-0.516795,0.052236,0.120773,1.225177,0.277302
4,0.0,-0.582831,-0.214671,-0.028993,-0.304770,-0.668038,-0.999288,0.277302
5,0.0,-0.133281,-0.077673,-0.178336,-0.534836,0.831580,0.266946,0.277302
...,...,...,...,...,...,...,...,...
361,0.0,-0.357181,-0.173572,-0.585302,-0.513300,0.281603,-0.639951,0.277302
362,0.0,-0.094570,-0.153022,-0.684802,-0.152980,0.110694,-0.400394,0.277302
363,0.0,-0.273340,-0.146172,-0.289763,-0.122125,-0.357334,-0.366171,0.277302
365,0.0,0.218035,-0.063973,0.101159,0.052236,0.380643,0.369614,0.277302


In [122]:
df.iloc[:,1:]=scale(df.iloc[:,1:])

##### On modelise

In [125]:
#Importation du model strocké dans un .pkl
model=pickle.load(open('model.pkl', 'rb'))

In [145]:
prediction= model.predict(df)

In [146]:
prediction

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,

In [147]:
df_pred=df
df_pred["Loan_Status_Predict"]= prediction

In [149]:
df_pred

,Never_Credit,Total_Income,Refund_Time,Income_pond,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Loan_Status_Predict
0,0.0,-0.107408,-0.153022,-0.157753,0.201334,-0.668038,-0.451727,0.277302,1
1,0.0,-0.329947,-0.125622,-0.570858,-0.346186,-0.010695,-0.177947,0.277302,1
2,0.0,0.102680,0.031925,-0.516795,0.052236,0.120773,1.225177,0.277302,1
4,0.0,-0.582831,-0.214671,-0.028993,-0.304770,-0.668038,-0.999288,0.277302,1
5,0.0,-0.133281,-0.077673,-0.178336,-0.534836,0.831580,0.266946,0.277302,1
...,...,...,...,...,...,...,...,...,...
361,0.0,-0.357181,-0.173572,-0.585302,-0.513300,0.281603,-0.639951,0.277302,1
362,0.0,-0.094570,-0.153022,-0.684802,-0.152980,0.110694,-0.400394,0.277302,1
363,0.0,-0.273340,-0.146172,-0.289763,-0.122125,-0.357334,-0.366171,0.277302,1
365,0.0,0.218035,-0.063973,0.101159,0.052236,0.380643,0.369614,0.277302,1


### Conclusion

On peut maintenant n'importe quel dataset avec les mêmes features pour utiliser notre modèle